<a href="https://colab.research.google.com/github/radicalcoder/ML-Project/blob/master/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import random
import matplotlib
import matplotlib.pyplot as plt
import skimage.data
import skimage.transform
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import itertools
from keras import models
from keras.preprocessing import image
from PIL import Image
from keras.models import Sequential
import keras
import PIL
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout


# To read the data directory from repository on github.
%matplotlib inline
!git clone -l -s https://github.com/radicalcoder/ML-Project.git data
%cd data
!ls

Cloning into 'data'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 7379 (delta 17), reused 0 (delta 0), pack-reused 7349
Receiving objects: 100% (7379/7379), 247.13 MiB | 27.23 MiB/s, done.
Resolving deltas: 100% (20/20), done.
Checking out files: 100% (7225/7225), done.
/content/data
BelgiumTSC_Testing   MLP_final_initialdraft.ipynb  README.md
BelgiumTSC_Training  MLP.ipynb			   traffic.py


In [0]:
def data_loader(data_dir):
    """
    The data set is read into two lists, as follows:
    list labels[] -> a list of numbers, where each number represents a unique image label.
    list images[] -> a list of arrays, where each array represents a single image.
    
    The data set directory contains multiple subdirectories. We read all these 
    subdirectories into a list subdirs[]. Since the subfolders are read in a random order, the list
    is also in a random order. We then sort this array using the sort() function.
    """
    
    subdirs = []
    for i in os.listdir(data_dir):
      if os.path.isdir(data_dir + i):   # There may be .txt files in the data_dir hence we need to check
        subdirs.append(i)

    subdirs.sort()
    
    """
    Data is now collected into two lists - labels & images - while looping through the 
    list subdirs[].    
    """

    labels = []
    images = []
    for i in subdirs:
        label_dir = data_dir + i
        #print(label_dir)
        f_names=[]
        for f in os.listdir(label_dir):
          if f.endswith(".ppm"):   # only files that end in .ppm are image files
            f_names.append(label_dir + '/' + f)
        # print(f_names)
        """
        Now for each label, append all its images into the list images[] and 
        append its label number in the list labels[]. 
        """
        
        for f in f_names:
            images.append(skimage.data.imread(f))   #use parameter as_grey??
            labels.append(int(i))
    return labels, images


# Loading the training and testing dataset using the function data_loader().
data_dir_training = "BelgiumTSC_Training/Training/"
data_dir_testing = "BelgiumTSC_Testing/Testing/"

labels, images = data_loader(data_dir_training)
#print(labels)

In [4]:
print("No of unique Labels: ", len(set(labels)), "\nTotal number of Images: ", len(images)) 


No of unique Labels:  62 
Total number of Images:  4575


In [0]:
reshape_images=[]
for i in images:
  reshape_images.append(skimage.transform.resize(i,(32,32), mode ='constant'))
  #print("shape: {0}, min: {1}, max: {2}".format(i.shape, i.min(), i.max()))

#for i in reshape_images[:20]:
  #print("shape: {0}, min: {1}, max: {2}".format(i.shape, i.min(), i.max()))